# Understanding MyGrad
> CogWorks 2018 (Petar Griggs)

In [1]:
# run this cell!
import mygrad as mg
import numpy as np

### Creating Tensors

Let's start by creating a list of numbers ranging from 0 to 50.

In [2]:
ls = list(range(51))

We can create a [`Tensor`](https://mygrad.readthedocs.io/en/latest/tensor.html)-instance by passing it some iterable of numbers. This includes lists, tuples, or NumPy arrays. Any array-like sequence of numbers will suffice. When we do so, the `Tensor.__init__()` simply converts this iterable to a NumPy `ndarray`. We can inspect a Tensor's underlying NumPy array by calling `<tensor>.data`. 

The takeaway is that: whatever data you pass in when creating a tensor will be stored as a NumPy array. You can access that underlying data **but should never unwittingly modify that NumPy array directly**.

Pass `ls` to `mg.Tensor` and check the shape and contents of the resulting tensor.

In [4]:
x = mg.Tensor(ls)
x

Tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

Check the `data` attribute of the Tensor that you created and see that it is the Tensor's underlying NumPy array.

In [5]:
x.data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

Try mutating the underlying NumPy array using an augmented assignment (e.g. `x += 2`), and see that this changes the Tensor correspondingly. This will come in handy when we are updating parameters via gradient descent.

In [7]:
x.data += 2
x

Tensor([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
        38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54])

Try constructing Tensors with other iterables below:
 - a 3D NumPy array
 - a 2D structure of nested lists
 - a single number (the only permissable non-iterable object that can be passed to `Tensor.__init__`)

In [9]:
x = mg.arange(1000).reshape(10, 10, 10)
x

Tensor([[[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19],
         [ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34,  35,  36,  37,  38,  39],
         [ 40,  41,  42,  43,  44,  45,  46,  47,  48,  49],
         [ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
         [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69],
         [ 70,  71,  72,  73,  74,  75,  76,  77,  78,  79],
         [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89],
         [ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99]],
 
        [[100, 101, 102, 103, 104, 105, 106, 107, 108, 109],
         [110, 111, 112, 113, 114, 115, 116, 117, 118, 119],
         [120, 121, 122, 123, 124, 125, 126, 127, 128, 129],
         [130, 131, 132, 133, 134, 135, 136, 137, 138, 139],
         [140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
         [150, 151, 152, 153, 154, 155, 156, 157, 158, 159],
         [160, 161, 1

MyGrad also has many convenient tensor-creation functions that mimic those of NumPy, such as [`arange`](https://mygrad.readthedocs.io/en/latest/generated/mygrad.arange.html#mygrad.arange) and [`linspace`](https://mygrad.readthedocs.io/en/latest/generated/mygrad.linspace.html#mygrad.linspace).

Execute `help(mg.tensor_creation.funcs)` to view these tensor-creation functions and their documentation, or see the [`mygrad` docs](https://mygrad.readthedocs.io/en/latest/tensor_creation.html). Notice that each of these functions accepts an optional `constant` argument. What do you suppose the purpose of this is?

Use `mg.linspace` to create a constant `Tensor` consisting of 100 points sampled on $[-1, 1]$. 

In [10]:
help(mg.tensor_creation.funcs)

Help on module mygrad.tensor_creation.funcs in mygrad.tensor_creation:

NAME
    mygrad.tensor_creation.funcs

FUNCTIONS
    arange(stop, start=0, step=1, dtype=None, constant=False)
        Return a Tensor with evenly-spaced values within a given interval.
        
        Values are generated within [start, stop). Note that for non-integer steps, results may be
        inconsistent; you are better off using `linspace` instead.
        
        Parameters
        ----------
        start : Real, optional, default=0
            The start of the interval, inclusive.
        
        stop : Real
            The end of the interval, exclusive.
        
        step : Real, optional (default=1)
            The spacing between successive values.
        
        dtype : data-type, optional (default=None)
            The data type of the output Tensor, or None to infer from the inputs.
        
        constant : bool, optional (default=False)
            If ``True``, the returned tensor is 

### Other Tensor Attributes
Tensors also store a number of other attributes: `scalar_only`, `grad`, `creator`, and `constant`. You can inspect the Tensor docstring for a description of some of these or read the docs [here](https://mygrad.readthedocs.io/en/latest/tensor.html#documentation-for-mygrad-tensor), but we will also describe them more in detail in due time.

### Creating Computational Graphs

As we perform mathematical operations using Tensors, we build up a computational graph whose nodes are Tensors and Operations. The edges connect tensor-nodes with operation-nodes.

Edges connect input-tensors to operations, and operations to output-tensors. Each new operation we perform adds new nodes and edges to our computational graph. Below is an example of a computational graph that you may remember from the online course, adapted slightly for MyGrad.

The following series of operations:

```python
x = mg.Tensor(10)
y = np.array(15)
out1 = x + y
z = 2
out2 = out1 * z
```

creates the computational graph:

![comp_graph](pics/comp_graph.png)

#### The creator attribute
Each tensor has a [`creator`](https://mygrad.readthedocs.io/en/latest/generated/mygrad.Tensor.creator.html#mygrad.Tensor.creator) attribute. This stores a reference to the Operation-instance that created that Tensor. `creator` will be `None` for any Tensor that was initialized directly, and not produced by a mathematical operation.

#### The variables attribute
Similarly, every Operation has a `variables` attribute, a tuple which stores all of the Tensors that acted as its inputs.

Together, `Tensor.creator` and `Operation.variables` are the two attributes required for defining the structure of any computational graph.

#### Tracing through a computational graph
Create the computational graph detailed above and inspect the creator-attribute of `out2`. Confirm that this is an instance of the `Multiplication` operation. Next, check the variables-attribute of that operation; it should store a tuple containing `out1` and `z`. Continue on until you've traced backward through the computational graph, back to `x` and `y`.

In [11]:
no u

SyntaxError: invalid syntax (<ipython-input-11-3e4e9d57395b>, line 1)

MyGrad must construct this computational graph so that it may use back-propagation to compute derivatives. More on this later. In the meantime it is critical to know that MyGrad is constructing these computational graphs "under the hood" whenever we perform mathematical operations on its Tensors.

You may be surprised to see that a NumPy-array was able to be used in your computational graph - MyGrad so closely mirrors NumPy that it is natural to permit the use of NumPy-arrays in its computational graphs. The key detail here is that *NumPy-arrays will always be treated as constants in computational graphs*. That is, if we perform back-propagation on this graph, no derivative will be computed for `y`, since it is a NumPy-array. This will prove to be very handy in the future.

### MyGrad's Math Library

To gain some familiarity with the mathematical functions supplied by mygrad, call `help` on: 
 - [`mg.math`](https://mygrad.readthedocs.io/en/latest/math.html)
 - [`mg.math.arithmetic.funcs`](https://mygrad.readthedocs.io/en/latest/math.html#arithmetic-operations)
 - [`mg.math.sequential.funcs`](https://mygrad.readthedocs.io/en/latest/math.html#sums-products-differences)
 - [`mg.linalg.funcs`](https://mygrad.readthedocs.io/en/latest/linalg.html)
 
All of these functions contained in these modules have a `backward` method, meaning that you can readily compute derivatives of them with respect to their inputs. More on this later.

Take some time to do some simple computations with mygrad's math functions. Note that they are all available at the top-level of mygrad (`mg.<TAB>` to reveal the functions available). If you are comfortable with calculus, you can try checking some derivatives out. For example:

```python
>>> x = mg.Tensor(0)
>>> mg.cos(x).backward() # computes the derivative d(cos(x))/dx
>>> x.grad # stores df/dx @ x = 0
array(-1.2246468e-16)
```